In [33]:
import pandas as pd

In [34]:
# Paste your OpenAI API key here
API_KEY = "YOUR_API_KEY"

In [35]:
# Read the dataset
df = pd.read_csv("NikeProductDescriptions.csv")
df.head()

,Title,Subtitle,Product Description
0,Nike Air Force 1 '07,Men's Shoes,It doesn't get more legendary than this. Desig...
1,Nike Air Max Dawn SE,Men's Shoes,Find out what moves you with the Air Max Dawn....
2,Nike SB Dunk Low Pro Premium,Skate Shoes,Pack your style—on your feet. Bringing a fresh...
3,Nike Air Force 1 Mid '07 LX,Men's Shoes,The celebrations just keep coming. Unbox the A...
4,Nike Air Force 1 Mid '07,Men's Shoes,"Got your fave colour yet? No worries, the Colo..."


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Title                400 non-null    object
 1   Subtitle             400 non-null    object
 2   Product Description  400 non-null    object
dtypes: object(3)
memory usage: 9.5+ KB


In [37]:
# Create Input column with concatenated Title and Subtitle
df['Input'] = 'Product: '+ df['Title']+" - " + df['Subtitle'] + "\n\nProduct Description:\n\n"

# Add " End" at the end of Product Descriptions
df['Output'] = " "+df['Product Description'] + " END"

In [38]:
print(df['Input'][25])

Product: Nike Air Max 97 SE - Men's Shoes

Product Description:




In [39]:
print(df['Output'][25])

 Find out what moves you in the Nike Air Max 97. Our iconic design, originally inspired by Japanese bullet trains, now tells a playful story rooted in our history of moving people. The textile upper, hemp accents and snail-trail stitching nod to our fictional "NIKE MOVING CO.". And to top it off on the bottom, visible Nike Air cushioning lets you ride in first-class comfort. So, where next? END


In [40]:
df = df[['Input', 'Output']]

In [41]:
df.columns = ['prompt','completion']

In [42]:
df.to_json("prepared_dataset.jsonl", orient='records', lines=True)

In [43]:
!pip install --upgrade openai -q

In [44]:
import os
os.environ['OPENAI_API_KEY'] = API_KEY

In [45]:
!openai tools fine_tunes.prepare_data -f prepared_dataset.jsonl

Analyzing...

- Your file contains 400 prompt-completion pairs
- All prompts end with suffix `\n\nProduct Description:\n\n`. This suffix seems very long. Consider replacing with a shorter suffix, such as `\n\n###\n\n`
- All prompts start with prefix `Product: `
- All completions end with suffix ` END`

No remediations found.

You can use your file for fine-tuning:
> openai api fine_tunes.create -t "prepared_dataset.jsonl"

After you’ve fine-tuned a model, remember that your prompt has to end with the indicator string `\n\nProduct Description:\n\n` for the model to start generating completions, rather than continuing with the prompt. Make sure to include `stop=[" END"]` so that the generated texts ends at the expected place.
Once your model starts training, it'll approximately take 7.94 minutes to train a `curie` model, and less for `ada` and `babbage`. Queue will approximately take half an hour per job ahead of you.


In [46]:
# !openai api fine_tunes.create -t "prepared_dataset.jsonl" -m 'davinci'

In [53]:
!openai api fine_tunes.follow -i "ft-k33aNArHOh2NoZFpnM1g6"

[2023-02-19 16:29:51] Created fine-tune: ft-k33aNAr6mHHOh2NoZFpnM1g6
[2023-02-19 16:33:30] Fine-tune costs $3.98
[2023-02-19 16:33:30] Fine-tune enqueued. Queue number: 1
[2023-02-19 16:33:32] Fine-tune started
[2023-02-19 16:37:40] Completed epoch 1/4
[2023-02-19 16:39:45] Completed epoch 2/4
[2023-02-19 16:41:50] Completed epoch 3/4
[2023-02-19 16:43:55] Completed epoch 4/4
[2023-02-19 16:44:35] Uploaded model: davinci:ft-indian-institute-of-information-technology-dharwad-2023-02-19-11-14-34
[2023-02-19 16:44:36] Uploaded result file: file-7qTyOQIM1FmPmiR52tattgxl
[2023-02-19 16:44:37] Fine-tune succeeded

Job complete! Status: succeeded 🎉
Try out your fine-tuned model:

openai api completions.create -m davinci:ft-indian-institute-of-information-technology-dharwad-2023-02-19-11-14-34 -p <YOUR_PROMPT>


In [73]:
prompt = "Product:  Nike Dunk Low Retro Premium - Men's Shoes\n\nProduct Description:\n\n"

In [74]:
# Test original model
import openai
openai.api_key = API_KEY
response = openai.Completion.create(
    model="davinci",
    prompt=prompt,
    max_tokens=100,
    temperature=0,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    stop=[" END"]
)
print(response['choices'][0]['text'])



The Nike Dunk Low Premium is a classic basketball shoe that features a leather upper with a padded ankle collar and a rubber outsole with herringbone pattern.

Product Details:

-Leather upper

-Padded ankle collar

-Rubber outsole with herringbone pattern

-Nike Dunk logo on tongue

-Nike Swoosh logo on side

-Nike Dunk logo on heel

-Nike Air logo on


In [75]:
# Test fine-tuned model
import openai
openai.api_key = API_KEY
response = openai.Completion.create(
    model="davinci:ft-indian-institute-of-information-technology-dharwad-2023-02-19-11-14-34",
    prompt=prompt,
    max_tokens=100,
    temperature=0,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    stop=[" END"]
)
print(response['choices'][0]['text'])


 The Nike Dunk Low Retro Premium delivers the iconic look you love, with premium details that honour the original design. Shiny leather adds a touch of luxury, while the honeycomb pattern on the heel adds a playful touch.
